In [62]:
import numpy as np
import h5py
from sklearn.utils import shuffle

from keras.layers import Conv2D,Dense,LeakyReLU,Dropout,Reshape,Flatten,MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam

In [63]:
bs = 128
total = 1012224
batches = total/bs
alpha = 0.0001

In [64]:
adam = Adam(lr = alpha)

In [65]:
model = Sequential()

model.add(Conv2D(6,kernel_size = 5,input_shape = (21,21,3),activation = 'relu'))
model.add(Conv2D(12,kernel_size = 5,activation = 'relu'))
model.add(Conv2D(16,kernel_size = 3,activation = 'relu'))
model.add(Conv2D(32,kernel_size = 3,activation = 'relu'))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))

In [66]:
model.summary()
model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 17, 17, 6)         456       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 13, 13, 12)        1812      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 11, 11, 16)        1744      
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 9, 9, 32)          4640      
_________________________________________________________________
flatten_8 (Flatten)          (None, 2592)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2593      
Total params: 11,245
Trainable params: 11,245
Non-trainable params: 0
_________________________________________________________________


In [46]:
with h5py.File("inputs.h5","r") as f:
    data = np.asarray(f['data'])
    label = np.asarray(f['label'])

In [47]:
label = label.reshape(total,1)
print(data.shape,label.shape)

(1012224, 21, 21, 3) (1012224, 1)


In [67]:
epochs = 10
data,label = shuffle(data,label)

Loss = []
Acc = []

In [68]:
for epoch in range(epochs):
    loss = 0
    acc = 0
    data,label = shuffle(data,label)
    print("################")
    print("Epoch:"+str(epoch+1))
    for i in range(int(batches)-1908):
        x_batch = data[bs*i:bs*(i+1),:,:,:]
        y_batch = label[bs*i:bs*(i+1),:]
        x_batch = x_batch/255
        
        l,a = model.train_on_batch(x_batch,y_batch)
        loss += l
        acc += a
        
        if i % 1000 == 999:
            print("Batch "+str(i+1)+" Completed")
            print("Loss:"+str(loss/(i+1)))    
            print("Accuracy:"+str(acc/(i+1)))
    print(model.evaluate(data[6000:7908,:,:,:]/255,label[6000:7908,:]))
    Loss.append(loss/(batches-1908))
    Acc.append(acc/(batches-1908))

################
Epoch:1
Batch 1000 Completed
Loss:0.6528079267144203
Accuracy:0.6267734375
Batch 2000 Completed
Loss:0.6393630838692188
Accuracy:0.64533984375
Batch 3000 Completed
Loss:0.6316034662127494
Accuracy:0.6525885416666667
Batch 4000 Completed
Loss:0.6248975602388382
Accuracy:0.65816796875
Batch 5000 Completed
Loss:0.6199541270136834
Accuracy:0.6621
Batch 6000 Completed
Loss:0.6164310218294462
Accuracy:0.6647213541666667
1908/1908 [==============================] - 1s 468us/step
[0.5960351602336396, 0.6792452828939106]
################
Epoch:2
Batch 1000 Completed
Loss:0.5945323075652122
Accuracy:0.681015625
Batch 2000 Completed
Loss:0.5921256026178598
Accuracy:0.68361328125
Batch 3000 Completed
Loss:0.5911314261257649
Accuracy:0.6844869791666667
Batch 4000 Completed
Loss:0.5900675772204995
Accuracy:0.685224609375
Batch 5000 Completed
Loss:0.589257286965847
Accuracy:0.6858359375
Batch 6000 Completed
Loss:0.5888444518943627
Accuracy:0.6860872395833333
1908/1908 [==============

In [70]:
print(model.evaluate(data[6000:7908,:,:,:]/255,label[6000:7908,:]))
print(Loss)
print(Acc)

1908/1908 [==============================] - 0s 215us/step
[0.5656451871560054, 0.7033542976939203]
[0.6164310218294462, 0.5888444518943627, 0.581217147871852, 0.5769679716974497, 0.5731658053894838, 0.5695581956207753, 0.5668258840243021, 0.5646734450260799, 0.5626291710635026, 0.560481743251284]
[0.6647213541666667, 0.6860872395833333, 0.692953125, 0.69688671875, 0.70028125, 0.70297265625, 0.7049453125, 0.7062434895833334, 0.7076888020833333, 0.7090755208333334]


In [72]:
model.save("models.h5")